<a href="https://colab.research.google.com/github/syedmahmoodiagents/RAG/blob/main/GraphNodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install langchain-openai langchain-huggingface --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 6.1 MB/s eta 0:00:00


In [1]:
from typing import TypedDict, List, Tuple
from langgraph.graph import StateGraph

In [12]:
import os, getpass

In [13]:
os.environ['HF_TOKEN'] = getpass.getpass()

··········


In [17]:
os.environ['OPENAI_API_KEY'] = getpass.getpass()

··········


In [18]:
from langchain_openai import ChatOpenAI
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

In [2]:
def traverse_graph(state):
    kg = state["knowledge_graph"]
    entities = state["entities"]

    facts = []
    for e in entities:
        for r, o in kg.edges.get(e, []):
            facts.append((e, r, o))

    state["graph_facts"] = facts
    return state

In [3]:

class KnowledgeGraph:
    def __init__(self):
        self.nodes = {}   # node_id -> attributes
        self.edges = {}   # node_id -> [(relation, target)]

    def add_node(self, node_id, **attrs):
        self.nodes[node_id] = attrs

    def add_edge(self, src, relation, dst):
        self.edges.setdefault(src, []).append((relation, dst))

    def neighbors(self, node_id):
        return self.edges.get(node_id, [])


In [4]:

kg = KnowledgeGraph()

kg.add_node("Aspirin", type="drug")
kg.add_node("COX-1", type="enzyme")
kg.add_node("Prostaglandins", type="chemical")
kg.add_node("Inflammation", type="effect")

kg.add_edge("Aspirin", "inhibits", "COX-1")
kg.add_edge("COX-1", "produces", "Prostaglandins")
kg.add_edge("Prostaglandins", "causes", "Inflammation")

In [5]:

class GraphRAGState(TypedDict):
    query: str
    knowledge_graph: KnowledgeGraph
    entities: List[str]
    graph_facts: List[Tuple[str, str, str]]
    context: str
    answer: str


In [6]:


def extract_entities(state: GraphRAGState) -> GraphRAGState:
    query = state["query"]
    kg = state["knowledge_graph"]

    entities = [
        e for e in kg.nodes.keys()
        if e.lower() in query.lower()
    ]

    state["entities"] = entities
    return state


In [7]:

def traverse_graph(state: GraphRAGState) -> GraphRAGState:
    kg = state["knowledge_graph"]
    entities = state["entities"]

    facts = []

    def dfs(node, depth, max_depth=3, visited=None):
        if visited is None:
            visited = set()
        if depth > max_depth or node in visited:
            return
        visited.add(node)

        for rel, nbr in kg.neighbors(node):
            facts.append((node, rel, nbr))
            dfs(nbr, depth + 1, max_depth, visited)

    for e in entities:
        dfs(e, 0)

    state["graph_facts"] = facts
    return state


In [8]:

def build_context(state: GraphRAGState) -> GraphRAGState:
    lines = [
        f"{s} {r} {o}."
        for s, r, o in state["graph_facts"]
    ]
    state["context"] = "\n".join(lines)
    return state


In [9]:
# Dummy LLM (replace with ChatOpenAI / ChatHuggingFace)
def answer_llm(state: GraphRAGState) -> GraphRAGState:
    prompt = f"""
Use ONLY the following facts to answer the question.

Facts:
{state['context']}

Question:
{state['query']}
"""

    # Replace this with: llm.invoke(prompt)
    state["answer"] = prompt.strip()
    return state

In [19]:
llm1 = ChatOpenAI(model='gpt-4o-mini')
llm2 = ChatHuggingFace(llm = HuggingFaceEndpoint(repo_id='openai/gpt-oss-20b'))

In [24]:
def answer_llm(state: GraphRAGState) -> GraphRAGState:
    prompt = f"""
Use ONLY the following facts to answer the question.

Facts:
{state['context']}

Question:
{state['query']}
"""

    # Replace this with: llm.invoke(prompt)
    # state["answer"] = prompt.strip()
    state["answer"] = llm2.invoke(prompt)
    return state

In [25]:
graph = StateGraph(GraphRAGState)

graph.add_node("extract_entities", extract_entities)
graph.add_node("traverse_graph", traverse_graph)
graph.add_node("build_context", build_context)
graph.add_node("answer", answer_llm)

graph.set_entry_point("extract_entities")
graph.add_edge("extract_entities", "traverse_graph")
graph.add_edge("traverse_graph", "build_context")
graph.add_edge("build_context", "answer")

app = graph.compile()

In [26]:


initial_state: GraphRAGState = {
    "query": "How does aspirin reduce inflammation?",
    "knowledge_graph": kg,
    "entities": [],
    "graph_facts": [],
    "context": "",
    "answer": ""
}

result = app.invoke(initial_state)

print("\n--- GRAPH CONTEXT ---")
print(result["context"])

print("\n--- FINAL ANSWER PROMPT ---")
print(result["answer"])



--- GRAPH CONTEXT ---
Aspirin inhibits COX-1.
COX-1 produces Prostaglandins.
Prostaglandins causes Inflammation.

--- FINAL ANSWER PROMPT ---
content='Aspirin reduces inflammation by inhibiting COX‑1, which in turn lowers the production of prostaglandins. Since prostaglandins cause inflammation, their reduced levels lead to less inflammation.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 119, 'total_tokens': 245}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_996f667773', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019bdeec-5bcf-7b30-9683-6d7ed9b824fa-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 119, 'output_tokens': 126, 'total_tokens': 245}
